In [27]:
# Setup environment
from pyspark.sql import DataFrame
from pyspark.sql import functions as F
from pyspark.sql.functions import split, col

StatementMeta(, 1c086541-1fc5-4156-81c1-40f4ca050b94, 29, Finished, Available, Finished)

In [28]:
# UDF to for checking null values counts in columns
def count_nulls(df: DataFrame):
    """
    Returns a Spark DataFrame showing the number of nulls per column
    for any input DataFrame.
    """
    null_counts = df.select([
        F.sum(F.col(c).isNull().cast("int")).alias(c)
        for c in df.columns
    ])
    return null_counts
   

StatementMeta(, 1c086541-1fc5-4156-81c1-40f4ca050b94, 30, Finished, Available, Finished)

In [29]:
# Read in data from delta tables
# table_of_interest = "NHL_Lakehouse_Bronze.nhl_game_plays" # Define table of interest (No corrections, direct bronze table)
table_of_interest = "NHL_Lakehouse_Silver.silver_events" # Define table of interest (Corrections applied, reused silver table)
df = spark.read.table(table_of_interest)


StatementMeta(, 1c086541-1fc5-4156-81c1-40f4ca050b94, 31, Finished, Available, Finished)

In [30]:
# Data exploration
df.show() 
df.printSchema() 



StatementMeta(, 1c086541-1fc5-4156-81c1-40f4ca050b94, 32, Finished, Available, Finished)

+--------------+----------+-----------+---------------+---------------+-------------+---+---+------+----------+----------+-------------------+-------------------+----------+----------+--------------------+----+----+
|       play_id|   game_id|team_id_for|team_id_against|          event|secondarytype|  x|  y|period|periodtype|periodtime|periodtimeremaining|           datetime|goals_away|goals_home|         description|st_x|st_y|
+--------------+----------+-----------+---------------+---------------+-------------+---+---+------+----------+----------+-------------------+-------------------+----------+----------+--------------------+----+----+
|2017020055_170|2017020055|         23|             52|    Missed Shot|           NA| 36| 15|     2|   REGULAR|      1092|                108|2017-10-13 04:35:15|         2|         1|Ben Hutton - Wide...|  36|  15|
|2017020055_171|2017020055|         52|             23|            Hit|           NA| 94| 34|     2|   REGULAR|      1095|              

# <u>Duplicate verification</u>
#### Can run this to find the duplicated rows

duplicate_rows = df.groupBy(df.columns)\
                   .agg(F.count("*").alias("count"))\
                   .filter(F.col("count") > 1)

duplicate_rows.show(truncate=False)

#### After showing the duplicated rows, can verify duplication with below code
df[df['play_id']=="2019020002_248"].show()

In [31]:
# Check for duplicate rows
total_rows = df.count()
duplicate_rows_count = df.count() - df.distinct().count()
print(f"Number of rows: {total_rows}")
print(f"Number of duplicate rows: {duplicate_rows_count}")

df_clean = df.dropDuplicates()

total_rows = df_clean.count()
duplicate_rows_count = df_clean.count() - df_clean.distinct().count()
print(f"\nNumber of rows (deduplicated): {total_rows}")
print(f"Number of duplicate rows (deduplicated): {duplicate_rows_count}")

StatementMeta(, 1c086541-1fc5-4156-81c1-40f4ca050b94, 33, Finished, Available, Finished)

Number of rows: 5050540
Number of duplicate rows: 833466

Number of rows (deduplicated): 4217074
Number of duplicate rows (deduplicated): 0


In [32]:
# List all columns
df_clean.columns

StatementMeta(, 1c086541-1fc5-4156-81c1-40f4ca050b94, 34, Finished, Available, Finished)

['play_id',
 'game_id',
 'team_id_for',
 'team_id_against',
 'event',
 'secondarytype',
 'x',
 'y',
 'period',
 'periodtype',
 'periodtime',
 'periodtimeremaining',
 'datetime',
 'goals_away',
 'goals_home',
 'description',
 'st_x',
 'st_y']

In [33]:
# Filter dataframe for interested columns
col_of_interest = ['play_id','game_id','event','period', 'periodtime', 'periodtype', 'datetime', 'goals_away', 'goals_home', 'description', 'st_x', 'st_y'] # Define columns of interest
df_clean = df_clean[col_of_interest]
df_clean.show()

StatementMeta(, 1c086541-1fc5-4156-81c1-40f4ca050b94, 35, Finished, Available, Finished)

+--------------+----------+--------------+------+----------+----------+-------------------+----------+----------+--------------------+----+----+
|       play_id|   game_id|         event|period|periodtime|periodtype|           datetime|goals_away|goals_home|         description|st_x|st_y|
+--------------+----------+--------------+------+----------+----------+-------------------+----------+----------+--------------------+----+----+
|2013021108_211|2013021108|          Shot|     2|       801|   REGULAR|2014-03-29 18:00:12|         3|         0|Reilly Smith Snap...|  67|  -5|
|2013021108_318|2013021108|          Shot|     3|       796|   REGULAR|2014-03-29 18:52:21|         3|         1|Dougie Hamilton W...|  48| -26|
|  2012020046_1|2012020046|Game Scheduled|     1|         0|   REGULAR|2013-01-24 23:19:17|         0|         0|      Game Scheduled|  NA|  NA|
| 2012020046_95|2012020046|       Faceoff|     1|      1081|   REGULAR|2013-01-25 02:08:22|         0|         0|Vernon Fiddler fa

In [34]:
# Split event seq number out to separate column for sorting purpose in play by play
df_clean = df_clean.withColumn(
    "event_seq",
    split(col("play_id"), "_").getItem(1).cast("int")
)

StatementMeta(, 1c086541-1fc5-4156-81c1-40f4ca050b94, 36, Finished, Available, Finished)

In [35]:
# Rename columns 
rename_dict = {
    "play_id"   : "event_id",
    "st_x"      : "xcord",
    "st_y"      : "ycord"
}

# Loop over the dictionary and rename columns
for old_name, new_name in rename_dict.items():
    df_clean = df_clean.withColumnRenamed(old_name, new_name)

# Show the updated DataFrame
df_clean.show(5)

StatementMeta(, 1c086541-1fc5-4156-81c1-40f4ca050b94, 37, Finished, Available, Finished)

+--------------+----------+--------------+------+----------+----------+-------------------+----------+----------+--------------------+-----+-----+---------+
|      event_id|   game_id|         event|period|periodtime|periodtype|           datetime|goals_away|goals_home|         description|xcord|ycord|event_seq|
+--------------+----------+--------------+------+----------+----------+-------------------+----------+----------+--------------------+-----+-----+---------+
|2013021108_211|2013021108|          Shot|     2|       801|   REGULAR|2014-03-29 18:00:12|         3|         0|Reilly Smith Snap...|   67|   -5|      211|
|2013021108_318|2013021108|          Shot|     3|       796|   REGULAR|2014-03-29 18:52:21|         3|         1|Dougie Hamilton W...|   48|  -26|      318|
|  2012020046_1|2012020046|Game Scheduled|     1|         0|   REGULAR|2013-01-24 23:19:17|         0|         0|      Game Scheduled|   NA|   NA|        1|
| 2012020046_95|2012020046|       Faceoff|     1|      108

In [36]:
# To view the distinct values in each column
column_name = 'game_id'
df_clean.select(column_name).distinct().count()
#df_clean.select(column_name).distinct().orderBy(column_name).show()

StatementMeta(, 1c086541-1fc5-4156-81c1-40f4ca050b94, 38, Finished, Available, Finished)

23323

# Sample of game with timedate inaccurate
df_clean[df_clean['game_id']==2000020001].orderBy('period','periodtime').show(df_clean.count(), truncate=False)

# Sample of game with accurate timedate 
df_clean[df_clean['game_id']==2011021158].orderBy('period','periodtime').show(df_clean.count(), truncate=False)

# Count of datetime duplicates by game_id
duplicates_df = (
    df_clean.groupBy("game_id", "datetime") 
            .agg(F.count("*").alias("count")) 
            .filter(F.col("count") > 1) 
            .orderBy("game_id", "datetime")
)
duplicates_df.show(1000)


# Remove rows with duplicate datetime
# Run if to ignore the duplicates
#df_clean[~df_clean'game_id'.isin(duplicates_df['game_id'].tolist)].show()

df_clean_no_dups = df_clean.join(
    duplicates_df.select("game_id").distinct(),
    on="game_id",
    how="left_anti"
)

df_clean_no_dups.show()

In [37]:
silver_events = (
    df_clean
    .withColumn("game_id", F.col("game_id").cast("string"))
    .withColumn("event_id", F.col("event_id").cast("string"))
    .withColumn("event", F.col("event").cast("string"))
    .withColumn("period", F.col("period").cast("byte"))
    .withColumn("periodtime", F.col("periodtime").cast("short"))
    .withColumn("datetime", F.col("datetime").cast("timestamp"))
    .withColumn("goals_away", F.col("goals_away").cast("byte"))
    .withColumn("goals_home", F.col("goals_home").cast("byte"))
    .withColumn("xcord", F.col("xcord").cast("byte"))
    .withColumn("ycord", F.col("ycord").cast("byte"))
)

StatementMeta(, 1c086541-1fc5-4156-81c1-40f4ca050b94, 39, Finished, Available, Finished)

In [38]:
silver_events.show()
silver_events.printSchema() 

StatementMeta(, 1c086541-1fc5-4156-81c1-40f4ca050b94, 40, Finished, Available, Finished)

+--------------+----------+--------------+------+----------+----------+-------------------+----------+----------+--------------------+-----+-----+---------+
|      event_id|   game_id|         event|period|periodtime|periodtype|           datetime|goals_away|goals_home|         description|xcord|ycord|event_seq|
+--------------+----------+--------------+------+----------+----------+-------------------+----------+----------+--------------------+-----+-----+---------+
|2013021108_211|2013021108|          Shot|     2|       801|   REGULAR|2014-03-29 18:00:12|         3|         0|Reilly Smith Snap...|   67|   -5|      211|
|2013021108_318|2013021108|          Shot|     3|       796|   REGULAR|2014-03-29 18:52:21|         3|         1|Dougie Hamilton W...|   48|  -26|      318|
|  2012020046_1|2012020046|Game Scheduled|     1|         0|   REGULAR|2013-01-24 23:19:17|         0|         0|      Game Scheduled| NULL| NULL|        1|
| 2012020046_95|2012020046|       Faceoff|     1|      108

In [39]:
count_nulls(silver_events).show(truncate=False)

StatementMeta(, 1c086541-1fc5-4156-81c1-40f4ca050b94, 41, Finished, Available, Finished)

+--------+-------+-----+------+----------+----------+--------+----------+----------+-----------+------+------+---------+
|event_id|game_id|event|period|periodtime|periodtype|datetime|goals_away|goals_home|description|xcord |ycord |event_seq|
+--------+-------+-----+------+----------+----------+--------+----------+----------+-----------+------+------+---------+
|0       |11     |0    |11    |11        |11        |11      |11        |11        |11         |975984|975984|0        |
+--------+-------+-----+------+----------+----------+--------+----------+----------+-----------+------+------+---------+



In [40]:
# Write cleaned data into lakehouse
(silver_events
    .write.format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable('NHL_Lakehouse_Silver.silver_events')
)

StatementMeta(, 1c086541-1fc5-4156-81c1-40f4ca050b94, 42, Finished, Available, Finished)

### Below code for verifying rows in records for missing event_ids
from pyspark.sql.functions import col  
missing_play_ids = [  
    "2011020767_23", "2012030135_359", "2013020459_258",  
    "2010020237_29", "2011021075_138", "2010020863_291",  
    "2011020642_5", "2015020367_33", "2018020963_83",  
    "2011030126_388", "2018020573_216"  
]

(  
    spark.table("NHL_Lakehouse_Silver.silver_events")  
         .filter(col("event_id").isin(missing_play_ids))  
         .show(truncate=False)  
)  